In [153]:
from math import ceil
import time, re, ast, sys, urllib, random, string, requests
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from selenium.webdriver.firefox.webdriver import FirefoxProfile
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
from itertools import cycle
from header_list import user_agent_list
from proxies_list import proxies_list_
from INITIAL_SCRAPPER_FUNCTIONS import *
from PARSER_FUNCTIONS import *
from LIST_DF_FUNCTIONS import *
from SEARCH_URL_GEN import *
from GET_SEARCH_URL import *
from GET_RESULTS import *
ua = user_agent_list
proxies = proxies_list_

In [154]:
len(proxies)

44

In [155]:
customer_url = input("Input the Redfin URL of the Home: ")

Input the Redfin URL of the Home: https://www.redfin.com/CA/San-Francisco/1100-Sacramento-St-94108/unit-708/home/51588892


In [156]:
header = random.sample(ua, 1)[0]
proxy = random.sample(proxies, 1)[0]
url = customer_url
print(proxy)
print(url)
soup = session_creator(proxy, ua, url)

104.165.219.164:46101
https://www.redfin.com/CA/San-Francisco/1100-Sacramento-St-94108/unit-708/home/51588892


In [157]:
customer_df = info_from_property(soup)

Index(['<span><a href="http://www.1100Sacramento708.com" rel="nofollow" target="_blank">Virtual Tour (External Link)</a></span>',
       'Virtual Media: ', '# of Spaces: ', '# of Garage Spaces: ',
       '<span>Designated On-Site Parking</span>', 'Parking Access: ',
       'Monthly Parking Fees: ', 'Parking Features: ',
       '<span>Gym, Garden/Greenbelt</span>',
       '<span>Transportation: 1 Block Away</span>',
       '<span>Shopping: 2 Blocks Away</span>', 'HOA Fee Includes: ',
       '<span>Has Homeowners Association</span>', 'Dues: ', '# of Bedrooms: ',
       'Bath Type/Includes: ', '# of Baths: ', '<span>Has View</span>',
       'Dining Room: ', 'Other Room(s): ', 'Den/Bonus Room: ',
       'Fireplace(s): ', '<span>Washer/Dryer, Laundry Room</span>',
       '<span>Hardwood, Tile</span>', 'Kitchen: ', 'Main Level: ',
       '<span>Elevator/Lift</span>', 'Financing Terms: ', 'Cross Street: ',
       'Original Price: ', 'On Market Date: ', 'Transfer of Possession: ',
       'View

In [160]:
customer_df.columns

Index(['address', 'city', 'state', 'zip_code', 'sq_ft', 'latitude',
       'longitude', 'price', 'num_bdrs', 'num_bts', 'redfin_est', 'on_redfin',
       'status', 'description', 'elem_school_name', 'elem_school_grades',
       'elem_school_rating', 'middle_school_name', 'middle_school_grades',
       'middle_school_rating', 'high_school_name', 'high_school_grades',
       'high_school_rating', 'beds', 'baths', 'finished_sq_ft',
       'unfinished_sq_ft', 'total_sq_ft', 'stories', 'lot_size', 'style',
       'year_built', 'year_renovated', 'county', 'apn',
       '<span><a_href="http://www1100sacramento708com"_rel="nofollow"_target="_blank">virtual_tour_external_link</a></span>',
       'virtual_media', 'num_spaces', 'num_garage_spaces',
       '<span>designated_on-site_parking</span>', 'parking_access',
       'monthly_parking_fees', 'parking_features',
       '<span>gym_garden/greenbelt</span>',
       '<span>transportation1_block_away</span>',
       '<span>shopping2_blocks_away</sp

In [163]:
a = [num for num in customer_df.columns if '<span>' in num]
[num.replace('<span>','').replace('</span>','') for num in a]

['<a_href="http://www1100sacramento708com"_rel="nofollow"_target="_blank">virtual_tour_external_link</a>',
 'designated_on-site_parking',
 'gym_garden/greenbelt',
 'transportation1_block_away',
 'shopping2_blocks_away',
 'has_homeowners_association',
 'has_view',
 'washer/dryer_laundry_room',
 'hardwood_tile',
 'elevator/lift',
 'per_architect',
 'tenancy_in_common_agreement',
 'limited_number_of_pets']

In [158]:
z = soup.findAll('span', {'class': re.compile('^header ')})
print([num.text for num in z])
a = soup.findAll('span', {'class': re.compile('^content ')})
print([num.text for num in a])

['HOA Dues', 'Property Type', 'Style', 'Community', 'County', 'MLS#', 'Built']
['$2,874/month', 'Hi-Rise (9+), Luxury, Tenancy In Common', 'Contemporary, Custom', 'Nob Hill', 'San Francisco', '476850', '1925']


In [159]:
c_url = gen_zip_url(customer_df)

KeyError: 1

In [ ]:
c_url

In [ ]:
header = random.sample(ua, 1)[0]
proxy = random.sample(proxies, 1)[0]
print(proxy)
soup = session_creator(proxy, ua, c_url)
get_results(soup, customer_df)